**ส่วนนี้คือการ load ข้อมูล และการกำจัด outliers (ท่านสามารถดูเพิ่มเติมได้ที่ 1_Data_Cleaning.ipynb)**

**ส่วนนี้ได้รวมการสร้าง dummy variables และการทำ data scaling (standardisation) ไว้ด้วย (ท่านสามารถดูเพิ่มเติมได้ที่ 3_Logit.ipynb)**

**สำหรับ Random Forest จะเริ่มต่อท้ายหลังจบส่วนนี้**

In [1]:
import sys
sys.version

'3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]'

In [2]:
# disable warnings that are not critical for a clean looking
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

import joblib

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
df = pd.read_csv('./dataset/Churn_Modelling.csv')

In [5]:
df.drop(['RowNumber','CustomerId','Surname'], axis=1, inplace=True)

#### การกำจัด outliers

In [6]:
def my_func(feature, upper, lower):
    tmp = df[(df[feature] >= upper) | (df[feature] <= lower)]
    df.drop(index=tmp.index, inplace=True)

In [7]:
my_func('CreditScore',None,383)
my_func('Age',62,None)

#### การสร้าง dummy variables

In [8]:
df_new = pd.get_dummies(df, drop_first=True)

#### ทำการแบ่งข้อมูล train/test ด้วยอัตราส่วน 80:20

In [9]:
X = df_new.drop('Exited', axis=1)
y = df_new['Exited']

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55)

#### Data Scaling (Standardisation)

In [11]:
from sklearn.preprocessing import StandardScaler

sc_X_train = StandardScaler()
sc_X_test = StandardScaler()

In [12]:
tmp = X_train
std = sc_X_train.fit_transform(tmp)
X_train = pd.DataFrame(std, columns=tmp.columns)

tmp = X_test
std = sc_X_test.fit_transform(tmp)
X_test = pd.DataFrame(std, columns=tmp.columns)

#### เขียนฟังก์ชั่นสำหรับประเมินประสิทธิภาพ model และเก็บค่าไว้ทำกราฟเปรียบเทียบในภายหลัง:

In [13]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

In [14]:
acc = {}; f1 = {}; prec = {}; recl = {}

def my_eval(model,y_test,predicted):
    
    print(model,'Results')
    print('Accuracy: ', accuracy_score(y_test, predicted))
    print('F1 Score: ', f1_score(y_test, predicted))
    print('Precision: ', precision_score(y_test, predicted))
    print('Recall: ', recall_score(y_test, predicted))
    
    acc[model] = accuracy_score(y_test, predicted)
    f1[model] = f1_score(y_test, predicted)
    prec[model] = precision_score(y_test, predicted)
    recl[model] = recall_score(y_test, predicted)
    
    # declair as a global model to ratain the evalutation values along the notebook
    global df_model_compare
    df_model_compare = pd.DataFrame.from_dict(
        [acc,f1,prec,recl]
    ).rename(
        index={0:'Accuracy',1:'F1 Score',2:'Precision',3:'Recall'}
    )
    #display(df_model_compare)

**Random Forest จะเริ่มต้นขึ้นต่อจากนี้**

---

# Random Forest

Decision Tree เป็น Weak Classifier  
Random Forest เป็น Strong Classifier

Random Forest เป็นการสร้าง Decision Tree หลาย ๆ ต้น จึงเป็น Ensemble Learning

Ensemble Learning คือการสร้าง Machine Learning หลาย ๆ model ให้มาช่วยกันตัดสิน ซึ่งช่วยป้องกันโมเดลตรรกะแปลก ๆ ที่ไม่ถูกต้องออก (protect each other from their individual errors) ทำให้เสียงส่วนใหญ่ไปในทิศทางที่ถูกต้อง โดยเมื่อโมเดลที่ไม่มีความเกี่ยวข้องกัน (uncorrelated) หลาย ๆ ตัวมาร่วมกันตัดสิน จะทำได้ดีกว่าโมเดลตัวเดี่ยว ๆ

เมื่อต้องการทำนายจะส่งข้อมูลไปให้โมเดลทุกตัวแล้วทำนายจากเสียงข้างมาก (majority vote)

In [15]:
from sklearn.ensemble import RandomForestClassifier

## การหาค่าที่เหมาะสมที่สุดด้วย Hyperparameter Tuning

In [16]:
from sklearn.model_selection import GridSearchCV

`max_depth` คือ ความลึกที่มากที่สุดของต้นไม้ (ถ้ามันอยากเกินที่เรากำหนด มันจะไปแตกก้านอื่นแทน) เช่น หาก max_depth = 1 คือแตกกิ่งจาก parent node จำนวน 1 ครั้งแล้วตัดสินเลย

`min_samples_leaf` คือ samples น้อยที่สุดที่ต้องการใน leaf node (node สุดท้ายก่อนตัดสินใจ) ยิ่งเงื่อนไขเยอะยิ่งมี node เยอะ

`n_estimators` = จำนวน Decision Trees ใน Random Forest  

In [17]:
parameters = {
    'max_depth':[4,12,20,40,None],
    'min_samples_leaf':[1,2,4,8,16,20],
    'min_samples_split':[2,4,8,16,32,64],
    'n_estimators':[10,50,100,300],
    'max_features':['sqrt', 'log2', None]
}

In [18]:
grid_search = GridSearchCV(RandomForestClassifier(random_state=55),
                           parameters,
                           verbose=10,
                           n_jobs=-1,
                           scoring='f1')

แสดง timestamp ก่อนรันอัลกอริทึม:

In [19]:
import datetime

timer_start = datetime.datetime.now()
print(datetime.datetime.now().time())

14:46:25.389574


สร้าง (ฝึกสอน) model:

In [20]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 2160 candidates, totalling 10800 fits


GridSearchCV(estimator=RandomForestClassifier(random_state=55), n_jobs=-1,
             param_grid={'max_depth': [4, 12, 20, 40, None],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 4, 8, 16, 20],
                         'min_samples_split': [2, 4, 8, 16, 32, 64],
                         'n_estimators': [10, 50, 100, 300]},
             scoring='f1', verbose=10)

แสดง timestamp หลังรันอัลกอริทึม:

In [21]:
timer_stop = datetime.datetime.now()
print(datetime.datetime.now().time())

16:13:52.788321


In [22]:
print('The time this algorithm utilised:',timer_stop - timer_start)

The time this algorithm utilised: 1:27:27.398747


**ค่าที่เหมาะสมที่สุด (optimised parameters):**

In [23]:
grid_search.best_params_

{'max_depth': 40,
 'max_features': None,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 50}

สามารถเลือกที่จะบันทึก model ที่จะสร้างใหม่ หรือจะโหลดจาก model ที่สร้างไว้แล้วก็ได้:

In [24]:
# Save Model
joblib.dump(grid_search, './models/RF.pkl')

# Load Model
#grid_search = joblib.load('./models/RF.pkl')

['./models/RF.pkl']

## การประเมิน model ที่ใช้ทำนาย

ใช้โมเดลทำนาย test set:

In [25]:
rf_predicted = grid_search.predict(X_test)

In [26]:
confusion_matrix(y_test, rf_predicted)

array([[1435,   71],
       [ 206,  203]], dtype=int64)

[ TN FP ]  
[ FN TP ]

TP (ทำนายว่า churn; ความจริงคือ churn)  
TN (ทำนายว่า non-churn; ความจริงคือ non-churn)  
FP (ทำนายว่า churn; ความจริงคือ non-churn)  
FN (ทำนายว่า non-churn; ความจริงคือ churn)

Churn = เลิกใช้ธนาคาร  
Non-churn = ไม่เลิกใช้ธนาคาร

เรียกฟังก์ชั่นประเมินประสิทธิภาพ model และเก็บค่าไว้ทำกราฟเปรียบเทียบในภายหลัง:

In [27]:
my_eval('RF', y_test, rf_predicted)

RF Results
Accuracy:  0.8553524804177546
F1 Score:  0.5944363103953149
Precision:  0.7408759124087592
Recall:  0.4963325183374083


---

บันทึกค่าประเมินประสิทธิภาพของแต่ละ model เพื่อใช้เปรียบเทียบใน notebook ชื่อ 9_Comparison.ipynb:

In [28]:
df_model_compare.to_csv('./eval_results/eval_results_RF.csv')